In [1]:
import numpy as np

In [6]:
def check_all_efx(assignments, vals):
    for j in range(len(assignments)):
        if not check_efx(assignments, vals, j):
            return False
    return True

def check_efx(assignments, vals, j):
    my_value = vals[j].dot(assignments[j])
    for i in range(len(assignments)):
        ass = assignments[i]
        ef_condition = vals[j].dot(ass) <= my_value
        if ef_condition:
            continue

        for l in range(len(ass)):
            if ass[l] > 0:
                new_vec = list(ass[:l]) + [ass[l] - 1] + list(ass[l+1:])
                condition = vals[j].dot(new_vec) <= my_value
                if not condition:
                    return False
    return True

def get_all_envy(assignments, vals):
    return np.array([get_envy(assignments, vals, j) for j in range(len(assignments))])

def get_envy(assignments, vals, j):
    my_value = vals[j].dot(assignments[j])
    envied = []
    for i in range(len(assignments)):
        ass = assignments[i]
        ef_delta = vals[j].dot(ass) - my_value
        envied.append(ef_delta)
    return envied

def get_envied_by(assignments, vals, i):
    envied_by = []
    ass = assignments[i]
    for j in range(len(assignments)):
        ef_delta = vals[j].dot(ass) - vals[j].dot(assignments[j])
        envied_by.append(ef_delta)
    return envied_by

def get_2_cycle(assignments, vals):
    arr = get_all_envy(assignments, vals) > 0
    for i in range(len(arr)):
        for j in range(len(arr)):
            if arr[i][j] and arr[j][i]:
                return i,j
    return False

In [10]:
def scale_inputs(valuations, items):
    valuations = np.array(valuations)

    top_prefs = np.argmax(valuations, axis=1)
    num_ppl = [sum(top_prefs == i) for i in range(len(items))]

    if items[0]/num_ppl[0] < items[1]/num_ppl[1]:
        vals = valuations[:,:]
    else:
        items = items[::-1]
        num_ppl = num_ppl[::-1]
        vals = valuations[:,::-1]
        
    # scale everything
    vals[:,0] /= vals[:,1]
    vals[:,1] = 1
    
    # change sort order
    order = np.argsort(vals[:,0])
    vals = vals[order, :][::-1, :]

    return vals, items

In [30]:
from itertools import product

In [46]:
from random import shuffle

In [142]:
def efx(vals, items, print_statements=False):
    pr = print if print_statements else lambda x: x
    n_players = len(vals)
    assignments = np.array([(0,0)]*n_players)
    items_left = np.copy(items)
    top_prefs = np.argmax(vals, axis=1)
    num_ppl = [sum(top_prefs == i) for i in range(len(items))]
    assert check_all_efx(assignments, vals), "NOT WORKING IN BEGINNING"
    
    # U0 for each type
    u0_applies = True
    while u0_applies:
        lst = list(product(range(2), range(n_players)))
        shuffle(lst)
#         pr(lst)
        u0_applies = False
        for t, i in lst:
            if items_left[t] > 0:
                test = np.copy(assignments)
                test[i][t] += 1
                if check_all_efx(test, vals):
                    assignments = test
                    items_left[t] -=1
                    u0_applies = True
                    pr(items_left)
#                     pr(assignments)
                    break

    pr("u0 done")
    # U2
    # find a source
    # give a good doesn't matter which
    # find minimal envied subset and player
    # get envied by player recurse until find a source
    # then repeat if source (or bundle?) not visited before...
    
    pr(items_left)
    assert check_all_efx(assignments, vals), "EFX SHOULD HOLD"
    assert sum(items_left) == 0, "SHOULD USE ALL ITEMS"
    assert np.all(np.sum(assignments, axis=0) == items), "ALL ITEMS ALLOCATED"
    
    return assignments

In [143]:
# t1, t2 valuations
orig_valuations = [
    (3,2),
    (2,1),
    (0.5,1),
    (4,1),
]

orig_items = [20, 20]

In [152]:
vals, items = scale_inputs(orig_valuations, orig_items)
print(vals, items)
efx(vals, items, print_statements=True)

[[4.  1. ]
 [2.  1. ]
 [1.5 1. ]
 [0.5 1. ]] [20, 20]
[20 19]
[19 19]
[18 19]
[18 18]
u0 done
[18 18]


AssertionError: SHOULD USE ALL ITEMS